In [ ]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LinearRegression   
import pickle

In [ ]:
df_delivery = pd.read_csv('../input/ipldata/deliveries.csv')

In [ ]:
df_matches = pd.read_csv('../input/ipldata/matches.csv')

In [ ]:
#remove unnecessary columns from df_delivery
removed_columns = ['batsman','non_striker','bowler','penalty_runs']
df_delivery.drop(labels=removed_columns, axis=1, inplace=True)

In [ ]:
#remove unnecessary columns from df_matches
removed_columns = ['umpire1','umpire2','umpire3','player_of_match']
df_matches.drop(labels=removed_columns, axis=1, inplace=True)

In [ ]:
df_matches.isnull().sum()

In [ ]:
rows_with_nan = []
for index, row in df_matches.iterrows():
    is_nan_series = row.isnull()
    if is_nan_series.any():
        rows_with_nan.append(index)
print(rows_with_nan)

In [ ]:
id_with_nan_in_matches = []
for val in rows_with_nan:
  id_with_nan_in_matches.append(df_matches.id[val])
print(id_with_nan_in_matches)

In [ ]:
#removing matches with nan result
index_names = df_delivery[ (df_delivery['match_id'] == 301) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 462) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 463) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 467) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 469) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 470) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 475) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 477) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 546) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 571) ].index
df_delivery.drop(index_names, inplace = True)
index_names = df_delivery[ (df_delivery['match_id'] == 11340) ].index
df_delivery.drop(index_names, inplace = True)

#for df_matches  
index_names = df_matches[ (df_matches['id'] == 301) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 462) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 463) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 467) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 469) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 470) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 475) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 477) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 546) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 571) ].index
df_matches.drop(index_names, inplace = True)
index_names = df_matches[ (df_matches['id'] == 11340) ].index
df_matches.drop(index_names, inplace = True)

In [ ]:
df_delivery.inning.unique()

In [ ]:
#removing unwanted innings
df_delivery.drop(df_delivery[df_delivery['inning'] >2].index, inplace = True)
#remove nann for dismissal fielder and player name
df_delivery["dismissal_kind"] = df_delivery["dismissal_kind"].fillna(0) 

In [ ]:
df_delivery.dismissal_kind.unique()

In [ ]:
for val in range(178830):
  if(val in df_delivery.index):
    if df_delivery.loc[val,'dismissal_kind'] == "bowled":
      df_delivery.loc[val,'dismissal_kind'] =1
    elif df_delivery.loc[val,'dismissal_kind'] == "caught":
      df_delivery.loc[val,'dismissal_kind'] =2
    elif df_delivery.loc[val,'dismissal_kind'] == "run out":
      df_delivery.loc[val,'dismissal_kind'] =3
    elif df_delivery.loc[val,'dismissal_kind'] == "caught and bowled":
      df_delivery.loc[val,'dismissal_kind'] =4
    elif df_delivery.loc[val,'dismissal_kind'] == "stumped":
      df_delivery.loc[val,'dismissal_kind'] =9
    elif df_delivery.loc[val,'dismissal_kind'] == "lbw":
      df_delivery.loc[val,'dismissal_kind'] =5
    elif df_delivery.loc[val,'dismissal_kind'] == "retired hurt":
      df_delivery.loc[val,'dismissal_kind'] =6
    elif df_delivery.loc[val,'dismissal_kind'] == "obstructing the field":
      df_delivery.loc[val,'dismissal_kind'] =7
    elif df_delivery.loc[val,'dismissal_kind'] == "hit wicket":
      df_delivery.loc[val,'dismissal_kind'] =8

In [ ]:
df_delivery['runs_till_now'] = 0
curr_inning = 0
prev_inning =0
start_inn_counter = 0
end_inning_counter =0
for x in range(1489):
  curr_inning = df_delivery.iloc[end_inning_counter,1]
  prev_inning = df_delivery.iloc[start_inn_counter,1]
  while curr_inning == prev_inning:
    end_inning_counter = end_inning_counter + 1
    curr_inning = df_delivery.iloc[end_inning_counter,1]
  df_delivery.iloc[start_inn_counter:end_inning_counter,17] = df_delivery.iloc[start_inn_counter:end_inning_counter,13].cumsum(axis =0)
  start_inn_counter = end_inning_counter
  prev_inning = curr_inning

In [ ]:
df_delivery['wicket_falled'] = 0
dismissed = 0
for val in range(178830):
  if(val in df_delivery.index):
    if df_delivery.loc[val,'dismissal_kind'] >0:
      df_delivery.loc[val,'wicket_falled'] =1
df_delivery['wickets_till_now'] = 0

In [ ]:
curr_inning = 0
prev_inning =0
start_inn_counter = 0
end_inning_counter =0
for x in range(1489):
  curr_inning = df_delivery.iloc[end_inning_counter,1]
  prev_inning = df_delivery.iloc[start_inn_counter,1]
  while curr_inning == prev_inning:
    end_inning_counter = end_inning_counter + 1
    curr_inning = df_delivery.iloc[end_inning_counter,1]
  df_delivery.iloc[start_inn_counter:end_inning_counter,19] = df_delivery.iloc[start_inn_counter:end_inning_counter,18].cumsum(axis =0)
  start_inn_counter = end_inning_counter
  prev_inning = curr_inning

In [ ]:
#new column for total run till the end of a perticular over
df_delivery['total_runs_in_over'] = 0

In [ ]:
curr_over = 1
next_over = 1
start_counter = 0
end_counter = 1
for x in range(23650):
  while curr_over == next_over:
    end_counter = end_counter + 1
    next_over = df_delivery.iloc[end_counter,4]
  if df_delivery.iloc[end_counter,1] == df_delivery.iloc[end_counter-1,1]:
    while end_counter != start_counter and (curr_over - next_over < 3):
      df_delivery.iloc[start_counter,20] = df_delivery.iloc[end_counter-1,17]
      start_counter = start_counter +1 
  curr_over = next_over
  start_counter = end_counter

In [ ]:
#removing last match data
df_delivery.drop(df_delivery[df_delivery['match_id'] >11414].index, inplace = True)
df_matches.drop(df_matches[df_matches['id'] >11414].index, inplace = True)

In [ ]:
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Pune Warriors'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Kochi Tuskers Kerala'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Deccan Chargers'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Pune Warriors'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Kochi Tuskers Kerala'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Deccan Chargers'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Gujarat Lions'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Gujarat Lions'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Rising Pune Supergiants'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Rising Pune Supergiant'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['bowling_team'] =='Rising Pune Supergiant'].index, inplace = True)
df_delivery.drop(df_delivery[df_delivery['batting_team'] =='Rising Pune Supergiants'].index, inplace = True)


In [ ]:
#merging both datasets
merged_inner = pd.merge(left=df_delivery, right=df_matches, left_on='match_id', right_on='id')

In [ ]:
removed_columns = ['wicket_falled','dismissal_kind','total_runs_in_over','result','winner' 	,'win_by_runs', 	'win_by_wickets'
                    ,'is_super_over','wide_runs','bye_runs','legbye_runs',
                   'noball_runs','batsman_runs','extra_runs',
                   'total_runs','player_dismissed','fielder','id',
                   'team1','team2','dl_applied','city']
merged_inner.drop(labels=removed_columns, axis=1, inplace=True)

In [ ]:
encoded_df = pd.get_dummies(data=merged_inner, columns=['batting_team', 'bowling_team','toss_winner'])
merged_inner = encoded_df

In [ ]:
#label encoding toss_decision season and venue
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
merged_inner['toss_decision'] = labelencoder.fit_transform(merged_inner['toss_decision'])
merged_inner['venue'] = labelencoder.fit_transform(merged_inner['venue'])
merged_inner['season'] = labelencoder.fit_transform(merged_inner['season'])

In [ ]:
#remove unnecessary columns from df_delivery
reserved_dataset = merged_inner
removed_columns = ['match_id','date']
reserved_dataset.drop(labels=removed_columns, axis=1, inplace=True)
final_dataset = reserved_dataset
new_dataset = final_dataset
final_dataset.head()

In [ ]:
train_x = final_dataset.iloc[:, [26,27,28,29,30,31,32,33,34,  8,9,10,11,12,13,14,15,16,    17,18,19,20,21,22,23,24,25,    0,1,2,4,5,6,7]]
train_y = final_dataset[["runs_till_now"]]

In [ ]:
"""
 #toss_winner_Chennai Super Kings 	toss_winner_Delhi Capitals 	toss_winner_Delhi Daredevils 	toss_winner_Kings XI Punjab 	toss_winner_Kolkata Knight Riders 	toss_winner_Mumbai Indians 	toss_winner_Rajasthan Royals 	toss_winner_Royal Challengers Bangalore 	toss_winner_Sunrisers Hyderabad  
 
 # batting_team_Chennai Super Kings 	batting_team_Delhi Capitals 	batting_team_Delhi Daredevils 	batting_team_Kings XI Punjab 	batting_team_Kolkata Knight Riders 	batting_team_Mumbai Indians 	batting_team_Rajasthan Royals 	batting_team_Royal Challengers Bangalore 	batting_team_Sunrisers Hyderabad 

 # bowling_team_Chennai Super Kings 	bowling_team_Delhi Capitals 	bowling_team_Delhi Daredevils 	bowling_team_Kings XI Punjab 	bowling_team_Kolkata Knight Riders 	bowling_team_Mumbai Indians 	bowling_team_Rajasthan Royals 	bowling_team_Royal Challengers Bangalore 	bowling_team_Sunrisers Hyderabad 	
"""

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(train_x,train_y, test_size =0.3, random_state=0)
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)
y_pred =regressor.predict(x_test)
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print(r2_score(y_pred,y_test)*100,'%')

In [ ]:
#testing purpose
main_x = [[ 0, 0, 1, 0, 0, 0, 0, 0, 0,     1, 0, 0, 0, 0, 0, 0, 0, 0,   1, 0, 0, 0, 0, 0, 0, 0, 0,   2, 5, 1, 5, 11, 1, 25]]

In [ ]:
print("predicted score for requestd over is", (regressor.predict(main_x)))

In [ ]:
"""
inning == (1,2)
over
ball
wickets_till_now
season = 11
toss_decision == (0==bat,1==field)
venue(0-37)

batting_team_Chennai Super Kings
batting_team_Delhi Capitals 	
batting_team_Delhi Daredevils
batting_team_Kings XI Punjab 	
batting_team_Kolkata Knight Riders
batting_team_Mumbai Indians 	
batting_team_Rajasthan Royals 	
batting_team_Royal Challengers Bangalore
batting_team_Sunrisers Hyderabad 	
bowling_team_Chennai Super Kings 	
bowling_team_Delhi Capitals 	
bowling_team_Delhi Daredevils 	
bowling_team_Kings XI Punjab 	
bowling_team_Kolkata Knight Riders 	
bowling_team_Mumbai Indians 	
bowling_team_Rajasthan Royals 	
bowling_team_Royal Challengers Bangalore 	
bowling_team_Sunrisers Hyderabad 	
toss_winner_Chennai Super Kings 	
toss_winner_Delhi Capitals 	
toss_winner_Delhi Daredevils 	
toss_winner_Kings XI Punjab 	
toss_winner_Kolkata Knight Riders 	
toss_winner_Mumbai Indians 	
toss_winner_Rajasthan Royals 	
toss_winner_Royal Challengers Bangalore 	
toss_winner_Sunrisers Hyderabad"""

In [ ]:
"""
Rajiv Gandhi International Stadium, Uppal', 25
'Maharashtra Cricket Association Stadium',  19
'Saurashtra Cricket Association Stadium', 
'Holkar Cricket Stadium', 11
'M Chinnaswamy Stadium',  15
'Wankhede Stadium',   35
'Eden Gardens',  7
'Feroz Shah Kotla',  8
'Punjab Cricket Association IS Bindra Stadium, Mohali',  23
'Green Park', 
'Punjab Cricket Association Stadium, Mohali',  24==23
'Sawai Mansingh Stadium',  28
'MA Chidambaram Stadium, Chepauk',  18
'Dr DY Patil Sports Academy',    5
'Newlands',  21
'St Georges Park', 23
'Kingsmead',  14
'SuperSport Park', 1
'Buffalo Park',   2
'Sardar Patel Stadium, Motera', 16
'Barabati Stadium',
'Vidarbha Cricket Association Stadium, Jamtha', 29
'Himachal Pradesh Cricket Association Stadium',   5
'Nehru Stadium',
'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
'Subrata Roy Sahara Stadium',
'Shaheed Veer Narayan Singh International Stadium',  2
'JSCA International Stadium Complex', 
'Sheikh Zayed Stadium',
'Sharjah Cricket Stadium', 
'M. A. Chidambaram Stadium',  16
'Feroz Shah Kotla Ground',   9
'M. Chinnaswamy Stadium',  15
'Rajiv Gandhi Intl. Cricket Stadium',  25
'IS Bindra Stadium',   12
'ACA-VDCA Stadium'  0"""

